# Reading Files into DataFrames

In [1]:
from pathlib import Path
import csv
import pandas as pd

### Use [pathlib](https://docs.python.org/3/library/pathlib.html) to get access to the files

`pathlib` is a module that provides classes representing filesystem paths with an OS agnostic interface. Here's an [in depth look at the library](https://youtu.be/UcKkmwaRbsQ). Note that because this notebook is in the same directory as the folder containing the data, we can use relative paths to access the data.

Note that there is no "mvsh4.dat" file, but you can still create a Path object for it. This is because Path objects are just a representation of a path, not the actual file itself.

In [2]:
mvsh1_file = Path("data/mvsh1.dat")
mvsh2_file = Path("data/mvsh2.dat")
mvsh3_file = Path("data/mvsh3.dat")
mvsh99_file = Path("data/mvsh99.dat")

One interesting thing about `Path` objects is that if you pass a `Path` object to `Path`, it will return the same `Path` object. This is useful for writing functions that can accept `Path` objects, strings representing paths, or a mix of both.

In [3]:
Path(mvsh1_file) == Path("data/mvsh1.dat")

True

We can check if the path is a file (`is_file()`), a directory (`is_dir()`), or if it exists (`exists()`).

In [4]:
mvsh1_file.is_file()

True

In [5]:
Path("data").is_dir()

True

In [6]:
mvsh99_file.exists()

False

Here we'll use `csvreader` to take a quick look at one of the .dat files. The `with` block of code is a form of Python's [context manager](https://realpython.com/python-with-statement/). In this case it handles some of the boilerplate code for opening and closing the file. We open the file but never explicitly close it; the context manager makes sure that whatever happens while we're reading the file, it will be closed when we're done.

We'll print each row of the file so we can take a look at the data, but more importantly we'll store the data in a list of lists called `mvsh1_csv`. Each row of the file is a list of strings, and the entire file is a list of those lists. We'll also track the number of lines in the file with `num_lines`.

In [7]:
mvsh1_csv: list[list[str]] = []
num_lines = 0
with mvsh1_file.open() as f:
    reader = csv.reader(f, delimiter="\t")
    for row in reader:
        # print(row) # comment out this line if you don't want to see the large block of output
        mvsh1_csv.append(row)
        num_lines += 1

The line defining `mvsh1_csv` includes type information, which is listed after a colon and before the equals sign. Python is a [dynamically typed language](https://realpython.com/python-type-checking/) and doesn't require type hints, but they are useful for documentation and gives tools like your IDE (e.g. VS Code), linters (e.g. `pylint`) and AI tools (e.g. [GitHub's Copilot](https://github.com/features/copilot)) more information to work with. Hover your cursor over `untyped_mvsh1_csv` (below) and `mvsh1_csv` to see the difference. Note that sometimes types can be automatically inferred, as in the case of `mvsh1_file` above.

VS Code will use type information when available to provide things like autocomplete and pop-up information about functions and variables. `pylance` -- a type checker within VS Code -- will also use type information to provide warnings about potential errors in the form of a squiggly line (red, yellow, or blue depending on the wanring type) under the text. If VS Code doesn't have enough information about the type of a variable, it uses the `Any` type and the text will be a different color than code with known type information.

For quick scripts that only you write, you don't need to use type hinting. I'll be using them throughout the tutorial because, in general, when writing code that other people will be working on, it's best to give them as much information as possible to understand the intent of the code.

In [8]:
untyped_mvsh1_csv = []

The number of lines is readily available in `num_lines`, but we don't actually need to track that ourselves. Instead we can use the `len()` method to inspect the length of the list.

In [9]:
num_lines

7346

In [10]:
num_lines == len(mvsh1_csv)

True

Making `mvsh1_csv` required two steps: first, make an empty list, and second, fill the list by going through the file line by line. That's pretty simple and could probably be done as is, but let's make it into a function so we can reuse it for the other files.

In [11]:
def dat_csv(file_path: str | Path) -> list[list[str]]:
    file_csv: list[list[str]] = []
    with Path(file_path).open() as f:
        reader = csv.reader(f, delimiter="\t")
        for row in reader:
            file_csv.append(row)
    return file_csv

Note the type hinting in the function definition. In the arguments of the function, `file_path` can be either a `str` or a `Path` (the vertical line `|` means "or"). The return type of the function is given after the arrow `->` and is a list of lists of strings. Again, typing is not required. That line could have been `def dat_csv(file_path):` and the function would work the same.

Below we'll use the function to read in the same file as before. Note the equivalence of the three versions of `mvsh1_csv`.

In [12]:
mvsh1_csv_2 = dat_csv(mvsh1_file)
mvsh1_csv_3 = dat_csv("data/mvsh1.dat")
mvsh1_csv == mvsh1_csv_2 == mvsh1_csv_3

True

With the file written into a convenient list we can take a look at some of the information in the header. Let's look at the 26th line (which is index 25 because Python lists are zero-indexed).

In [13]:
mvsh1_csv[25]

['INFO',
 '1566.22',
 'SAMPLE_MOLECULAR_WEIGHT',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

Each line has a lot of empty values, likely because Quantum Design used a fixed width determined by the longest line in the file. Let's just look at a slice of the previous list to see only the relevant values.

In [14]:
mvsh1_csv[25][0:3]

['INFO', '1566.22', 'SAMPLE_MOLECULAR_WEIGHT']

### Using `pandas` to read and use the data

The actual data in a .dat file begins after the line containing "[Data]". In this case the header is 40 lines long. The `pandas` library is probably the most used library in data science to organize data. The main class we'll be using is the `DataFrame`. `DataFrame` objects are like spreadsheets -- they have rows and columns, each column has a name, and we can access the data in each column by name and specific rows by index values. There are a lot of additional features that we'll get to later.

Reading in plain text data in a csv-type format is made easy with the `read_csv()` function. We'll use the `skiprows` argument to skip the first 40 rows of the file, and the `sep` argument to specify that the data is separated by tabs. The names of the columns can be manually entered or, in this case, taken from the first row of the data section of the file.

In [15]:
mvsh1_df = pd.read_csv(mvsh1_file, sep="\t", skiprows=40)

Take a look at the first 5 rows of the data using the `head()` method.

In [16]:
mvsh1_df.head()

,Comment,Time Stamp (sec),Temperature (K),Magnetic Field (Oe),Moment (emu),M. Std. Err. (emu),Transport Action,Averaging Time (sec),Frequency (Hz),Peak Amplitude (mm),...,Map 07,Map 08,Map 09,Map 10,Map 11,Map 12,Map 13,Map 14,Map 15,Map 16
0,NaN,3803627317,2.000165,70000.37500,0.736924,0.000996,1,1,13.006381,0.999015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,3803627320,2.000241,69995.39844,0.736522,0.001055,1,1,13.006381,0.999028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3803627325,1.999892,69746.85938,0.737400,0.001470,1,1,13.006381,0.999024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3803627334,2.000141,69286.15625,0.736039,0.000992,1,1,13.006381,0.999066,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,3803627335,1.999827,69246.48438,0.737444,0.001020,1,1,13.006381,0.999066,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Exercises

The files `mvsh1_file`, `mvsh2_file`, and `mvsh3_file` are all M vs H measurements, but one of them has a different number of lines in the header. 

### 1.1

Write a function `get_header()` which takes in a string or path to a .dat file and returns a dataframe with the data. To do this you'll want to first write a function that returns just the header information.

Remember that in .dat files, the header starts with a line that just says "[Header]" and the data begins after a line that says "[Data]". You'll need to look into [how to use `break`, `continue`, and `pass` to control the flow of a loop.](https://www.geeksforgeeks.org/break-continue-and-pass-in-python/) The last line of the header should be the one containing "[Data]".

In [17]:
def get_header(file_path: str | Path) -> list[list[str]]:
    header: list[list[str]] = []
    with Path(file_path).open() as f:
        reader = csv.reader(f, delimiter="\t")
        for row in reader:
            header.append(row)
            if row[0] == "[Data]":
                break
    return header

With `assert` statements, if the expression after `assert` returns `True` then nothing happens, otherwise it raises an `AssertionError`. We'll use them to check your work against the correct answer. If you get an error in the cells containing `assert` statements, change your function and try again.

In [18]:
assert len(get_header(mvsh1_file)) == 40
assert len(get_header("data/mvsh1.dat")) == 40
assert len(get_header(mvsh2_file)) == 40
assert len(get_header(mvsh3_file)) == 27

### 1.2

Now write a function `get_data()` which takes in a string or path to a .dat file and returns a dataframe with the data. You can use the `get_header()` function you wrote above to get the header information.

In [19]:
def get_data(file_path: str | Path) -> pd.DataFrame:
    file_path = Path(file_path)
    skip_rows = len(get_header(file_path))
    return pd.read_csv(file_path, sep="\t", skiprows=skip_rows)

In [20]:
assert len(get_data(mvsh1_file)) == 7305
assert len(get_data("data/mvsh1.dat")) == 7305
assert len(get_data(mvsh2_file)) == 719
assert len(get_data(mvsh3_file)) == 141

Now that your `get_data()` functions are working, let's make some dataframes for each file.

In [21]:
mvsh1_df = get_data(mvsh1_file)
mvsh2_df = get_data(mvsh2_file)
mvsh3_df = get_data(mvsh3_file)

### 1.3

`mvsh1` contains data from a VSM measurement while `mvsh2` and `mvsh3` contain data from a DC measurement. Quantum Design keeps the same column names regardless of the measurement. The VSM column names for magnetic moment are "Moment (emu)" and "M. Std. Err. (emu)". The DC column names (at least the ones that we'll be using) are "DC Moment Free Ctr (emu)" and "DC Moment Err Free Ctr (emu)".

Write a function `measurement_type()` which takes in a dataframe and returns a string "VSM" or "DC" depending on the measurement type. Return an empty string ("") if the measurement type is unknown. You'll need the [`notna()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.notna.html#pandas.Series.notna) and [`all()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.all.html) methods to check if an entire column is empty. Assume that the person who collected the data is civilized and only performed one type of measurement in the file.

In [22]:
def measurement_type(df: pd.DataFrame) -> str:
    meas_type = ""
    if df["Moment (emu)"].notna().all():
        meas_type = "VSM"
    elif df["DC Moment Free Ctr (emu)"].notna().all():
        meas_type = "DC"
    return meas_type

In [23]:
assert measurement_type(mvsh1_df) == "VSM"
assert measurement_type(mvsh2_df) == "DC"
assert measurement_type(mvsh3_df) == "DC"

### 1.4

We'd like to be able to easily work with dc data without having to worry about which measurement method was used. Also, for now we only care about the following columns: "Time Stamp (sec)", "Temperature (K)", "Magnetic Field (Oe)", and columns for the magnetic moment and its error. And since we'd like to be lazy when working with the data later, let's rename the columns to something shorter.

Write a function `simplify_magnetic_df()` which takes in a dataframe containing the magnetic data from a .dat file and returns a dataframe with the columns: "time", "temp", "field", "moment", and "moment_err".

In [24]:
def simplify_magnetic_df(df: pd.DataFrame) -> pd.DataFrame:
    new_df = pd.DataFrame()
    new_df["time"] = df["Time Stamp (sec)"]
    new_df["temp"] = df["Temperature (K)"]
    new_df["field"] = df["Magnetic Field (Oe)"]
    meas_type = measurement_type(df)
    if meas_type == "VSM":
        new_df["moment"] = df["Moment (emu)"]
        new_df["moment_err"] = df["M. Std. Err. (emu)"]
    elif meas_type == "DC":
        new_df["moment"] = df["DC Moment Free Ctr (emu)"]
        new_df["moment_err"] = df["DC Moment Err Free Ctr (emu)"]
    return new_df

In [25]:
assert set(simplify_magnetic_df(mvsh1_df).columns.tolist()) == {"time", "temp", "field", "moment", "moment_err"}
assert simplify_magnetic_df(mvsh1_df)["moment"].notna().all()
assert set(simplify_magnetic_df(mvsh2_df).columns.tolist()) == {"time", "temp", "field", "moment", "moment_err"}
assert simplify_magnetic_df(mvsh2_df)["moment"].notna().all()

Now that these are probably working as intended, let's make some simplified dataframes for each file.

In [26]:
mvsh1_simple = simplify_magnetic_df(mvsh1_df)
mvsh2_simple = simplify_magnetic_df(mvsh2_df)
mvsh3_simple = simplify_magnetic_df(mvsh3_df)

`mvsh3` contains data from a single temperature sweep, while `mvsh1` and `mvsh2` contain data from multiple temperature sweeps. We'll save the parsing of multiple temperature sweeps for later, so for now we'll focus on `mvsh3`.

### 1.5

Write a function `get_avg_temp()` which takes in a simplified magnetic dataframe and returns the average temperature of the measurement.

In [27]:
def get_avg_temp(df: pd.DataFrame) -> float:
    return df["temp"].mean()

In [28]:
assert round(get_avg_temp(mvsh3_simple), 2) == 5.00

### 1.6

Write a function `get_elapsed_time()` which takes in a simplified magnetic dataframe and returns the elapsed time of the measurement in minutes (90 seconds would be 1.5 minutes).

In [29]:
def get_elapsed_time(df: pd.DataFrame) -> int:
    return (df["time"].max() - df["time"].min())/60

In [30]:
assert get_elapsed_time(mvsh3_simple) == 63.15

### 1.7

Write a function `get_remanance()` which takes in a simplified magnetic dataframe and returns a list of two-tuples where each tuple contains the field and moment at the field. The field should be the field closest to zero and the moment should be the moment at that field. The fields are never exactly set to the fields listed in the sequence, so when searching for the zero field you can round to the nearest 10 (but still report the actual field value). 

For example, if during the field sweep the field is set to [..., -500, -250, 0, 250, 500, ...] but the actual fields are [..., -501.3, -249.2, 1.2, 249.8, 499.9, ...], the function should return (for this sweep) the tuple `(1.2, X)`, where X is the moment at 1.2 Oe.

Note: this is no longer a function that can be coded with a single method from `pandas`

In [31]:
def get_remanance(df: pd.DataFrame) -> list[tuple[float, float]]:
    df["rounded_field"] = df["field"].round(-2)
    zero_field_indexes = df[df["rounded_field"] == 0].index.tolist()
    remanance = []
    for i in zero_field_indexes:
        remanance.append((df["field"][i], df["moment"][i]))
    return remanance

In [32]:
assert get_remanance(mvsh3_simple) == [(-0.354434133, -0.015702303), (0.244437352, -0.000278623)]